# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [4]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
customer_products.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [10]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

N_Clients = 1000000
N_Products = 500000
print(N_Clients*N_Products*8/1024/1024/1024/1024)
prod_cust_pivot.head()

3.637978807091713


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [11]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [14]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.015873,0.000000,0.000000,0.0,0.0,0.015873,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.015873,0.0
200,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.012987,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.012987,0.000000,0.0
264,0.0,0.0,0.0,0.0,0.000000,0.015385,0.015385,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.015385,0.0,0.0,0.0,0.000000,0.000000,0.0
356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014925,0.0
412,0.0,0.0,0.0,0.0,0.013699,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.013699,0.013699,0.013699,0.0,0.0,0.0,0.000000,0.000000,0.0


## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [28]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)
cust_dist_cos = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'cosine'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

display(cust_dist.head())
print(cust_dist.loc[33].sort_values(ascending=False).head())
print(cust_dist_cos.loc[33].sort_values(ascending=False).head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


CustomerID
33       1.000000
60862    0.870544
27672    0.867011
6001     0.864672
79458    0.864671
Name: 33, dtype: float64
CustomerID
33       1.000000
27672    0.575128
45549    0.574835
3531     0.573871
76532    0.573713
Name: 33, dtype: float64


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [30]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [32]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [33]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [34]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [36]:
prod_dist_cos = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot.T, 'cosine'))),
                         index=cust_prod_pivot.T.index, columns=cust_prod_pivot.T.index)

prod_dist_cos.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.532644,0.529683,0.537361,0.543908,0.518135,0.533064,0.529602,0.552230,0.551982,...,0.549088,0.555667,0.541419,0.528259,0.526952,0.561138,0.533673,0.543279,0.535951,0.534201
"Appetizer - Mini Egg Roll, Shrimp",0.532644,1.000000,0.530024,0.536176,0.540399,0.528373,0.531902,0.547134,0.543714,0.532629,...,0.538812,0.538839,0.541315,0.531346,0.529652,0.527741,0.533817,0.545132,0.538357,0.535723
Appetizer - Mushroom Tart,0.529683,0.530024,1.000000,0.534145,0.521837,0.525393,0.530842,0.526655,0.536472,0.523479,...,0.529625,0.534931,0.539900,0.548054,0.539790,0.532205,0.535488,0.527628,0.533547,0.522834
Appetizer - Sausage Rolls,0.537361,0.536176,0.534145,1.000000,0.528917,0.533184,0.524616,0.527457,0.521652,0.527560,...,0.533456,0.546518,0.544733,0.553248,0.523528,0.518962,0.543771,0.534935,0.530126,0.525558
Apricots - Dried,0.543908,0.540399,0.521837,0.528917,1.000000,0.536647,0.524616,0.534440,0.531307,0.540414,...,0.529254,0.533402,0.540122,0.541369,0.524434,0.521778,0.542475,0.531635,0.541357,0.530317


### Step 2: Get the products purchased for a specific customer of your choice.

In [38]:
prod_cust_pivot[33].sort_values(ascending=False).head()

ProductName
Grouper - Fresh                0.001229
Veal - Inside, Choice          0.001098
Lettuce - Spring Mix           0.001012
Salsify, Organic               0.000906
Water - Green Tea Refresher    0.000756
Name: 33, dtype: float64

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [101]:
df = []
for customer in cust_prod_pivot.index:
    top_prods_by_customer = prod_cust_pivot[customer].sort_values(ascending=False).head()
    top_related_product_for_product = pd.Series([])
    for product in top_prods_by_customer.index:
       top_related_product_for_product = pd.concat([top_related_product_for_product,prod_dist_cos[product].sort_values(ascending=False)[1:].head()])
    top_related_product_for_product = top_related_product_for_product.sort_values(ascending=False)
    not_acquired = prod_cust_pivot[customer] == 0
    top_related_product_for_product = top_related_product_for_product[prod_cust_pivot[customer].isin(not_acquired)]
    top_by_customer_related_products = pd.Series(list(top_related_product_for_product.index.unique())).head()
    df.append([customer, *top_by_customer_related_products])
df = pd.DataFrame(df, columns=["Customer","P1","P2","P3","P4","P5"]).
display(df)

,Customer,P1,P2,P3,P4,P5
0,33,Meldea Green Tea Liquor,Ezy Change Mophandle,"Cheese - Boursin, Garlic / Herbs",Bread - Multigrain,Loquat
1,200,Wine - Two Oceans Cabernet,Chocolate - Feathers,Wine - Chablis 2003 Champs,Tuna - Salad Premix,Ocean Spray - Ruby Red
2,264,Ezy Change Mophandle,Tilapia - Fillets,Spice - Peppercorn Melange,Scallops - 10/20,Halibut - Steaks
3,356,Broom - Corn,Beans - Kidney White,Arizona - Green Tea,"Cheese - Boursin, Garlic / Herbs",Watercress
4,412,Wine - Prosecco Valdobiaddene,Beef - Montreal Smoked Brisket,Cream Of Tartar,Soup - Campbells Tomato Ravioli,"Chocolate - Semi Sweet, Calets"
5,464,Sauce - Rosee,Puree - Passion Fruit,Otomegusa Dashi Konbu,"Oregano - Dry, Rubbed",Broom - Corn
6,477,Crackers - Trio,Bouq All Italian - Primerba,Pasta - Orecchiette,Cornflakes,"Pasta - Penne, Rigate, Dry"
7,639,Wine - Vidal Icewine Magnotta,Cookies - Assorted,Puree - Passion Fruit,"Bread - Roll, Soft White Round",Tea - Earl Grey
8,649,Wine - Pinot Noir Latour,"Salmon - Atlantic, Skin On",Bread Crumbs - Japanese Style,Cheese - Taleggio D.o.p.,"Nut - Chestnuts, Whole"
9,669,Beer - Rickards Red,Flavouring - Orange,Coffee - Hazelnut Cream,Raspberries - Fresh,Banana - Leaves


### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [102]:
df = df.set_index("Customer")
display(df)

,P1,P2,P3,P4,P5
Customer,,,,,
33,Meldea Green Tea Liquor,Ezy Change Mophandle,"Cheese - Boursin, Garlic / Herbs",Bread - Multigrain,Loquat
200,Wine - Two Oceans Cabernet,Chocolate - Feathers,Wine - Chablis 2003 Champs,Tuna - Salad Premix,Ocean Spray - Ruby Red
264,Ezy Change Mophandle,Tilapia - Fillets,Spice - Peppercorn Melange,Scallops - 10/20,Halibut - Steaks
356,Broom - Corn,Beans - Kidney White,Arizona - Green Tea,"Cheese - Boursin, Garlic / Herbs",Watercress
412,Wine - Prosecco Valdobiaddene,Beef - Montreal Smoked Brisket,Cream Of Tartar,Soup - Campbells Tomato Ravioli,"Chocolate - Semi Sweet, Calets"
464,Sauce - Rosee,Puree - Passion Fruit,Otomegusa Dashi Konbu,"Oregano - Dry, Rubbed",Broom - Corn
477,Crackers - Trio,Bouq All Italian - Primerba,Pasta - Orecchiette,Cornflakes,"Pasta - Penne, Rigate, Dry"
639,Wine - Vidal Icewine Magnotta,Cookies - Assorted,Puree - Passion Fruit,"Bread - Roll, Soft White Round",Tea - Earl Grey
649,Wine - Pinot Noir Latour,"Salmon - Atlantic, Skin On",Bread Crumbs - Japanese Style,Cheese - Taleggio D.o.p.,"Nut - Chestnuts, Whole"


### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [ ]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

### Step 8: Recommend 5 products to this new customer using a item similarity approach.